In [ ]:
# search jira for issues created in specific time range
# and write them to a csv file
# usage: python3 jira_search.py <start_date> <end_date> <output_file>
# example: python3 jira_search.py 2020-01-01 2020-12-31 jira_issues.csv


In [ ]:
import requests
import csv
import sys

def search_jira_issues(start_date, end_date, output_file):
    # Jira server and authentication details
    jira_server = 'https://your-jira-server.com'
    auth = ('your_username', 'your_password')

    # JQL query to search for issues created in the specified time range
    jql_query = f'created >= "{start_date}" AND created <= "{end_date}"'
    search_url = f'{jira_server}/rest/api/2/search'

    start_at = 0
    max_results = 100
    all_issues = []

    while True:
        # Search for issues with pagination
        params = {
            'jql': jql_query,
            'startAt': start_at,
            'maxResults': max_results,
            'fields': 'key,summary,status,created,updated'  # Optimize by requesting only needed fields
        }
        
        response = requests.get(search_url, auth=auth, params=params)
        
        if response.status_code != 200:
            print(f"Error: Unable to fetch issues. Status code: {response.status_code}, Response: {response.text}")
            return

        data = response.json()
        issues = data['issues']
        total = data['total']
        
        if not issues:
            break
            
        all_issues.extend(issues)  # More efficient than +=
        print(f"Fetched {len(all_issues)} of {total} issues")
        
        if len(all_issues) >= total:
            break
            
        start_at += max_results

    # Write issues to CSV file
    with open(output_file, mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Issue Key', 'Summary', 'Status', 'Created', 'Updated'])

        for issue in all_issues:
            writer.writerow([
                issue['key'],
                issue['fields']['summary'],
                issue['fields']['status']['name'],
                issue['fields']['created'],
                issue['fields']['updated']
            ])

    print(f"Successfully exported {len(all_issues)} issues to {output_file}")